### Base model

Our base model is a plain ResNet50 model which is initialized with random weights and therefore has no prior knowledge of any images.

In [1]:
from keras import preprocessing, layers, Sequential
from keras.applications import ResNet50
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.resnet50 import preprocess_input

C:\Users\lodu\anaconda3\envs\dev\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
random_seed = 42
np.random.seed(random_seed)

In [6]:
train_data_dir = './data_split/train'
test_data_dir = './data_split/test'

img_size = (224, 224)
batch_size = 32

train_images = preprocessing.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

val_images = preprocessing.image_dataset_from_directory(
    test_data_dir,
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_images.class_names

train_ds = train_images.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_images.map(lambda x, y: (preprocess_input(x), y))

Found 1545 files belonging to 3 classes.
Found 517 files belonging to 3 classes.


In [ ]:
# Build model from scratch (random init)
base_model = ResNet50(include_top=True, weights=None, input_shape=(224, 224, 3), classes=len(class_names)) # include_top=True adds the classification layer

base_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = base_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

In [ ]:
def plot_loss(history):
    plt.plot(history['loss'], label='loss')
    plt.plot(history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_loss(history.history)

In [ ]:
def plot_accuracy(history):
    plt.plot(history['accuracy'], label='accuracy')
    plt.plot(history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

plot_accuracy(history.history)

In [ ]:
test_loss, test_accuracy = base_model.evaluate(val_ds)

print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

### Transfer learning

### Data augmentation

In [8]:
zoom_aug = Sequential([
    layers.RandomZoom(0.2),
])

random_zoom_train = train_images.map(lambda x, y: (preprocess_input(zoom_aug(x)), y))




In [9]:
crop_aug = Sequential([
    layers.RandomCrop(224, 224),
])

increased_size_train = preprocessing.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size=(256, 256),
    batch_size=batch_size,
    shuffle=False
)

random_crop_train = increased_size_train.map(lambda x, y: (preprocess_input(crop_aug(x)), y))




Found 1545 files belonging to 3 classes.


In [11]:
translate_aug = Sequential([
    layers.RandomTranslation(0.2, 0.2),
])

random_translate_train = train_images.map(lambda x, y: (preprocess_input(translate_aug(x)), y))




### Different architectures

### Test on online images

### Conclusion